In [2]:
import cv2 as cv
from os import listdir
import preprocess_lego_image
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
import tensorflow as tf
tf.config.list_physical_devices('GPU')

Note that Keras now has a lot more functionality than when I initially wrote some of the below code. Loading data is now much easier with [`image_dataset_from_directory`](https://keras.io/api/data_loading/image/). There are also many [code examples](https://keras.io/examples/) involving image classification that will probably work better than the below code. However, this will give a good introduction and a surprisingly accurate model (at least for my data). 

In [3]:
data_dir_labeled = '/data/labeled'
labels = listdir(data_dir_labeled)
labels.sort()
labels

In [4]:
def load_data():
    data = []
    for label in labels:
        label_path = f'{data_dir_labeled}/{label}'
        image_paths = listdir(label_path)
        for image_path in image_paths:                
            image = preprocess_lego_image.preprocess(f'{label_path}/{image_path}')
            data.append([label, image])
    return data

labeled_data = load_data()

In [5]:
plt.figure(figsize=(10, 10))
i = 0
for label, image in random.sample(labeled_data, 9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(label)
    plt.axis("off")
    i = i + 1

In [6]:
df = pd.DataFrame(labeled_data,columns = ['label','image'])
data_train = df.sample(frac=0.8, random_state=1)
data_test = df.drop(data_train.index)
print(f'# training data: {len(data_train)}')
print(f'# training data: {len(data_test)}')

In [7]:
from tensorflow.keras.utils import to_categorical
X_train = np.array(data_train.drop(['label'], axis=1))
y_train = to_categorical(np.array(data_train['label'].apply(lambda l: labels.index(l))))
X_test = np.array(data_test.drop(['label'], axis=1))
y_test = to_categorical(np.array(data_test['label'].apply(lambda l: labels.index(l))))

In [8]:
#sanity check
random_index = random.randint(0, len(X_train) - 1)
plt.title(labels[np.argmax(y_train[random_index])])
plt.imshow(cv.cvtColor(X_train[random_index][0], cv.COLOR_BGR2RGB))

In [9]:
L = []
for k in X_train:
    L += [k[0]]
L = np.array(L)
X_train = L
X_train.shape

In [10]:
P = []
for k in X_test:
    P += [k[0]]
P = np.array(P)
X_test = P
X_test.shape

In [11]:
X_test[0].shape

In [12]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
X_test = preprocess_input(X_test)
X_train = preprocess_input(X_train)
X_test.shape

In [26]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))
for layer in vgg16.layers:
    layer.trainable = False
vgg16.summary()

In [27]:
x = tf.keras.layers.Flatten()(vgg16.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(len(labels), activation='softmax')(x)

In [28]:
model = tf.keras.models.Model(vgg16.input, output)

In [29]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [30]:
model.build([None, 256, 256, 3])
model.summary()

In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=200, verbose=1)

In [32]:
plt.figure(figsize=(10, 10))
i = 0
for label, image in random.sample(labeled_data, 9):
    ax = plt.subplot(3, 3, i + 1)
    expected_label, example_image = random.sample(labeled_data, 1)[0]
    prediction = model.predict(preprocess_input(np.array([example_image])))
    most_likely = np.argmax(prediction)
    predicted_label = labels[most_likely]
    predicted_prob = prediction[0][most_likely]
    plt.title(f'{expected_label}\n{predicted_label}\n{predicted_prob}')
    plt.imshow(cv.cvtColor(example_image, cv.COLOR_BGR2RGB))
    plt.axis("off")
    i = i + 1

In [34]:
# fine tuning
for layer in vgg16.layers:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [36]:
plt.figure(figsize=(10, 10))
i = 0
for label, image in random.sample(labeled_data, 9):
    ax = plt.subplot(3, 3, i + 1)
    expected_label, example_image = random.sample(labeled_data, 1)[0]
    prediction = model.predict(preprocess_input(np.array([example_image])))
    np.argmax(prediction)
    predicted_label = labels[np.argmax(prediction)]
    plt.title(f'{expected_label}\n{predicted_label}')
    plt.imshow(cv.cvtColor(example_image, cv.COLOR_BGR2RGB))
    plt.axis("off")
    i = i + 1

In [37]:
model.save('model.keras')

We now have a trained model that is stored in `model.keras`. In the next notebook, we will use this model to classify images and convert it into a REST API that can be used by the Raspberry Pi controlling the LEGO sorter.